# Project 01. Funnel Analysis

In [236]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.tight_layout()

<Figure size 432x288 with 0 Axes>

# Table of Contents
  1. [Read Data](#readdata)
  2. [Funnel Conversion Rate](#funnelconversionrate)
  3. [Other Insights](#otherinsights)

<a id='read_data'></a>
## Read Data

In [237]:
# Since 'user_id' is the key of all the tables, change all the index in tables to be 'user_id'.
data = pd.read_csv('C:/Users/xiaoj/Desktop/CS/collection of data science-take home challenges/Data/5. Funnel_Analysis/user_table.csv', index_col='user_id')
data.head()

,date,device,sex
user_id,,,
450007,2015-02-28,Desktop,Female
756838,2015-01-13,Desktop,Male
568983,2015-04-09,Desktop,Male
190794,2015-02-18,Desktop,Female
537909,2015-01-15,Desktop,Male


In [238]:
home = pd.read_csv('C:/Users/xiaoj/Desktop/CS/collection of data science-take home challenges/Data/5. Funnel_Analysis/home_page_table.csv',index_col='user_id')
search = pd.read_csv('C:/Users/xiaoj/Desktop/CS/collection of data science-take home challenges/Data/5. Funnel_Analysis/search_page_table.csv',index_col='user_id')
pay = pd.read_csv('C:/Users/xiaoj/Desktop/CS/collection of data science-take home challenges/Data/5. Funnel_Analysis/payment_page_table.csv',index_col='user_id')
confirmation = pd.read_csv('C:/Users/xiaoj/Desktop/CS/collection of data science-take home challenges/Data/5. Funnel_Analysis/payment_confirmation_table.csv',index_col='user_id')

In [239]:
# add one column 'page' to the data to show the status 
data.loc[home.index,'page'] = home['page']
data.loc[search.index,'page'] = search['page']
data.loc[pay.index,'page'] = pay['page']
data.loc[confirmation.index, 'page'] = confirmation['page']
data.head()

,date,device,sex,page
user_id,,,,
450007,2015-02-28,Desktop,Female,home_page
756838,2015-01-13,Desktop,Male,home_page
568983,2015-04-09,Desktop,Male,search_page
190794,2015-02-18,Desktop,Female,search_page
537909,2015-01-15,Desktop,Male,home_page


In [240]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90400 entries, 450007 to 438929
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    90400 non-null  object
 1   device  90400 non-null  object
 2   sex     90400 non-null  object
 3   page    90400 non-null  object
dtypes: object(4)
memory usage: 5.9+ MB


In [241]:
#change the date from object to datetime
data['date']=pd.to_datetime(data['date'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90400 entries, 450007 to 438929
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    90400 non-null  datetime64[ns]
 1   device  90400 non-null  object        
 2   sex     90400 non-null  object        
 3   page    90400 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 5.9+ MB


<a id='funnelconversionrate'></a>
## Funnel Conversion Rate

In [242]:
#define a new dataframe to show the stay, convert_rate etc.
def convert_rate(data):
    summary = pd.DataFrame()
    number = data['page'].value_counts()
    summary['stay'] = number
    summary['total'] = data.shape[0],search.shape[0],pay.shape[0],confirmation.shape[0]
    summary['convertion_rate'] = (summary['stay'])/summary['total']
    return summary

In [243]:
data.groupby('device').apply(convert_rate)

stay  total  convertion_rate
device                                                          
Desktop home_page                  30100  60200         0.500000
        search_page                27090  45200         0.599336
        payment_page                2860   6030         0.474295
        payment_confirmation_page    150    452         0.331858
Mobile  home_page                  15100  30200         0.500000
        search_page                12080  45200         0.267257
        payment_page                2718   6030         0.450746
        payment_confirmation_page    302    452         0.668142

<a id='otherinsights'></a>
## Other Insights

In [244]:
# check the influence of time
data_=data.copy()
data_['weekday']=data['date'].dt.day_name()
data_['page']= (data['page']=='payment_confirmation_page').astype(int)

data_.head()

,date,device,sex,page,weekday
user_id,,,,,
450007,2015-02-28,Desktop,Female,0,Saturday
756838,2015-01-13,Desktop,Male,0,Tuesday
568983,2015-04-09,Desktop,Male,0,Thursday
190794,2015-02-18,Desktop,Female,0,Wednesday
537909,2015-01-15,Desktop,Male,0,Thursday


In [267]:
weekday = data_.groupby('weekday')['page'].agg(['count','mean']).sort_values(by='mean')
weekday.loc['Average'] = weekday.mean()
weekday['Difference'] = weekday.

,count,mean
weekday,,
Wednesday,12831.000000,0.004209
Friday,12750.000000,0.004235
Sunday,12922.000000,0.004798
Thursday,13444.000000,0.004909
Tuesday,12697.000000,0.005434
Saturday,12801.000000,0.005468
Monday,12955.000000,0.005944
Average,12914.285714,0.005000


In [262]:
# check the device influence
device = data_.groupby('device')['page'].agg(['count','mean']).sort_values(by='mean')
device.loc['Difference']=(device.loc['Mobile']-device.loc['Desktop'])/device.loc['Desktop']
device

,count,mean
device,,
Desktop,60200.000000,0.002492
Mobile,30200.000000,0.010000
Difference,-0.498339,3.013333


In [263]:
# check the sex influence
sex = data_.groupby('sex')['page'].agg(['count','mean']).sort_values(by='mean')
sex.loc['Difference']=(sex.loc['Male']-sex.loc['Female'])/sex.loc['Male']
sex

,count,mean
sex,,
Male,45325.000000,0.004655
Female,45075.000000,0.005347
Difference,0.005516,-0.148515


Summary:
1. The count of using mobile is less than 50% of that of desktop, but the conversion rate is more than 3 times than that of desktop. 
2. The male and female has almost the same counts, but the conversion rate of Female is almost 15% higher than that of Male.